In [1]:
import plotly.express as px
import pandas as pd
import streamlit as st
from warehouse import find_num_contributions_using_name

In [8]:
data = find_num_contributions_using_name("Sun Aixin")
data = data[0]
def plot_year_of_involvement(data):
    years = sorted([int(year) for year in data['num_contributions'].keys()])
    contribution_types = set()
    for year_data in data['num_contributions'].values():
        contribution_types.update(year_data.keys())

    data_list = []
    for year, year_data in data['num_contributions'].items():
        row = {'Year': int(year)}
        row.update(year_data)
        data_list.append(row)

    df = pd.DataFrame(data_list)

    # Define a custom color scale for contribution types
    custom_color_scale = {
        'Journal Articles': '#aec7e8',  # Light blue
        'Conference and Workshop Papers': '#ff7f0e',  # Orange
        'Informal and Other Publications': '#1f77b4',  # Dark blue
        'Editorship': '#ffbb78',  # Light orange
        'Books and Theses': '#98df8a',  # Light green
        'Parts in Books or Collections': '#d62728',  # Red
        'Reference Works': '#c5b0d5'  # Lavender
    }

    # Create the stacked bar chart
    fig = px.bar(df, x='Year', y=list(contribution_types), title=f'{data["name"]} - Year of Involvement',
                 labels={'value': 'Number of Contributions', 'variable': 'Contribution Type'},
                 barmode='relative',color='variable', color_discrete_map=custom_color_scale)

    # Show the chart
    fig.update_layout(xaxis_title='Year', yaxis_title='Number of Contributions',
                     plot_bgcolor='white',paper_bgcolor='white')
    
    # Display the Plotly chart in Streamlit
    st.title("Stacked Bar Chart - Year of Involvement")
    st.plotly_chart(fig)

In [40]:
import networkx as nx
import plotly.graph_objects as go

# Data
data = {
    'name': 'Li Fang',
    'coauthors': [
        {'coauthor_name': 'Maylor K. H. Leung', 'times': 7},
        {'coauthor_name': 'Xiaozhou Yu', 'times': 1},
        {'coauthor_name': 'Namisha Khanna', 'times': 1},
        {'coauthor_name': 'Purva Satija', 'times': 1},
        {'coauthor_name': 'Shirley Z. W. Tan', 'times': 1},
        {'coauthor_name': 'Chuan Liu', 'times': 1},
        {'coauthor_name': 'Cheng Shao Chian', 'times': 1}
    ]
}

# Create a directed graph
G = nx.DiGraph()

# Add nodes and edges
G.add_node(data['name'], size=20, color='orange')  # Professor node in orange
times_collaborated_total = 0

for coauthor in data['coauthors']:
    coauthor_name = coauthor['coauthor_name']
    times_collaborated = coauthor['times']
    times_collaborated_total += times_collaborated  # Accumulate total collaborations
    # Adjust node size based on times_collaborated
    node_size = 10 + 5 * times_collaborated
    G.add_node(coauthor_name, size=node_size, color='grey')  # Coauthor nodes in grey
    G.add_edge(data['name'], coauthor_name, times=times_collaborated)  # Add 'times' as an edge attribute

# Use a spring layout for a force-directed layout with shorter edges
pos = nx.spring_layout(G, seed=42, scale=2)

# Create node positions
node_x = []
node_y = []
node_sizes = []
node_colors = []
node_labels = []

for node, (x, y) in pos.items():
    node_x.append(x)
    node_y.append(y)
    node_sizes.append(G.nodes[node]['size'])
    node_colors.append(G.nodes[node]['color'])
    edge_data = G.get_edge_data(data['name'], node)  # Corrected access to edge attributes
    if edge_data is not None:
        times_collaborated = edge_data['times']
    else:
        times_collaborated = times_collaborated_total
    node_labels.append(f"{node}<br>Collaborations: {times_collaborated}")

# Create edge positions
edge_x = []
edge_y = []

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

# Create a Plotly figure
fig = go.Figure()

# Add edges
fig.add_trace(go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='gray'),
    hoverinfo='none',
    mode='lines'))

# Add nodes
fig.add_trace(go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    text=node_labels,
    marker=dict(
        size=node_sizes,
        color=node_colors,
        line=dict(width=2, color='black'),  # Node border
    )))

# Set layout options
fig.update_layout(
    title='Professor Li Fang\'s Coauthor Network',
    showlegend=False,
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),  # Remove axis
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),  # Remove axis
    plot_bgcolor='rgba(0,0,0,0)'  # Remove background
)

# Show the plot
fig.show()
